In [13]:
# FP树
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
    
    # 增加计数
    def inc(self, numOccur):
        self.count += numOccur
        
    # 展示函数
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)

In [11]:
rootNode = treeNode('pyramid', 9, None)
rootNode.children['eye'] = treeNode('eye', 13, None)

rootNode.disp()

   pyramid   9
     eye   13


In [12]:
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)
rootNode.disp()

   pyramid   9
     eye   13
     phoenix   3


In [40]:
# FP树的构建
def createTree(dataSet, minSup=1):
    headerTable = {}
    for trans in dataSet:  # 第一次遍历数据集并统计频次
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable.keys()):  # 少于最小支持度的删除
        if headerTable[k] < minSup:
            del headerTable[k]
    freqItemSet = set(headerTable.keys())
    if len(freqItemSet) == 0:  # 如果没有频繁集则退出
        return None, None
    
    # 对头指针表进行扩展，保存计数值及指向每种类型的第一个元素项的指针
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
        
    retTree = treeNode('Null Set', 1, None)
    for tranSet, count in dataSet.items():
        localD = {}
        for item in tranSet:  # 第二次遍历只考虑频繁集
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:  # 降序排序后更新
            orderedItems = [v[0] for v in sorted(localD.items(), \
                                                 key=lambda p: p[1], reverse=True)]
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:  # 如果是子节点则增加计数
        inTree.children[items[0]].inc(count)
    else:  # 非子节点则创建节点并更新指向
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
            
    if len(items) > 1:  # 存在多个项则递归更新
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

# 确保节点链接指向树中该元素项的每一个实例
def updateHeader(nodeToTest, targetNode):
    while nodeToTest.nodeLink != None:
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [41]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
              ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
              ['z'],
              ['r', 'x', 'n', 'o', 's'],
              ['y', 'r', 'x', 'z', 'q', 't', 'p'],
              ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

# 将dataSet项集频次设置为1并以字典形式返回
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [42]:
simpDat = loadSimpDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [43]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [46]:
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

   Null Set   1
     z   5
       r   1
       x   3
         s   2
           y   2
             t   2
         y   1
           t   1
             r   1
     x   1
       s   1
         r   1


In [47]:
## 创建条件模式基
# 上溯FP树
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

# 遍历链表直到结尾
def findPrefixPath(basePat, treeNode):
    condPats = {}  # 条件模式基字典
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink  # 相关节点
    return condPats

In [48]:
findPrefixPath('x', myHeaderTab['x'][1])

{frozenset({'z'}): 3}

In [49]:
findPrefixPath('z', myHeaderTab['z'][1])

{}

In [50]:
findPrefixPath('r', myHeaderTab['r'][1])

{frozenset({'z'}): 1,
 frozenset({'s', 'x'}): 1,
 frozenset({'t', 'x', 'y', 'z'}): 1}

In [74]:
for p in myHeaderTab.items():
    print('===================================')
    print(p)
    print(p[1])
    print(p[1][0])

('z', [5, <__main__.treeNode object at 0x000001AEF3FC28C8>])
[5, <__main__.treeNode object at 0x000001AEF3FC28C8>]
5
('r', [3, <__main__.treeNode object at 0x000001AEF3FC2E88>])
[3, <__main__.treeNode object at 0x000001AEF3FC2E88>]
3
('s', [3, <__main__.treeNode object at 0x000001AEF484C988>])
[3, <__main__.treeNode object at 0x000001AEF484C988>]
3
('y', [3, <__main__.treeNode object at 0x000001AEF484C4C8>])
[3, <__main__.treeNode object at 0x000001AEF484C4C8>]
3
('t', [3, <__main__.treeNode object at 0x000001AEF47E9BC8>])
[3, <__main__.treeNode object at 0x000001AEF47E9BC8>]
3
('x', [4, <__main__.treeNode object at 0x000001AEF3FC29C8>])
[4, <__main__.treeNode object at 0x000001AEF3FC29C8>]
4


In [78]:
# 递归查找频繁项集
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    # 按频次升序得到项集列表
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]
#     print('bigL: ', bigL)  # 如：['r', 's', 'y', 't', 'x', 'z']
    
    for basePat in bigL:
        newFreqSet = preFix.copy()  # 复制原频繁项集
        newFreqSet.add(basePat)  # 添加频繁项集元素
        freqItemList.append(newFreqSet)  # 添加频繁项集
        
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])  # 构建条件基
        myCondTree, myHead = createTree(condPattBases, minSup)  # 创建条件FP树
        if myHead != None:
#             print('conditional tree for: ', newFreqSet)
#             myCondTree.disp(1)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [76]:
freItems = []  # 频繁项集
mineTree(myFPtree, myHeaderTab, 3, set([]), freItems)

conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'y', 'x'}
   Null Set   1
     z   3
conditional tree for:  {'t'}
   Null Set   1
     y   3
       z   3
         x   3
conditional tree for:  {'z', 't'}
   Null Set   1
     y   3
conditional tree for:  {'t', 'x'}
   Null Set   1
     z   3
       y   3
conditional tree for:  {'y', 't', 'x'}
   Null Set   1
     z   3
conditional tree for:  {'x'}
   Null Set   1
     z   3


In [77]:
freItems

[{'r'},
 {'s'},
 {'s', 'x'},
 {'y'},
 {'y', 'z'},
 {'x', 'y'},
 {'x', 'y', 'z'},
 {'t'},
 {'t', 'y'},
 {'t', 'z'},
 {'t', 'y', 'z'},
 {'t', 'x'},
 {'t', 'x', 'z'},
 {'t', 'x', 'y'},
 {'t', 'x', 'y', 'z'},
 {'x'},
 {'x', 'z'},
 {'z'}]

## 实例：从新闻网站点击流中挖掘

In [80]:
# 导入数据集
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
parsedDat[:3]

[['1', '2', '3'], ['1'], ['4', '5', '6', '7']]

In [81]:
# 初始化数据
initSet = createInitSet(parsedDat)
# 构建FP树，并寻找至少被10万人浏览过的新闻报道
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [82]:
# 保存频繁集项
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)

In [83]:
# 查看有多少新闻被10万人浏览过
len(myFreqList)

9

In [84]:
# 查看具体新闻标号
myFreqList

[{'1'},
 {'1', '6'},
 {'3'},
 {'11', '3'},
 {'11', '3', '6'},
 {'3', '6'},
 {'11'},
 {'11', '6'},
 {'6'}]